In [1]:
import sys
sys.path.append("..")
from MPRA_predict.utils import *
from MPRA_predict.datasets import *
from MPRA_predict.metrics import *

In [2]:
MPRA_df = pd.read_csv('../data/Reddy_MPRA/Reddy_MPRA_Sei_feature_0507.csv')
MPRA_df

,seq,Jurkat,K562,THP-1,class,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_H3K27me3,K562_H3K4me1,K562_H3K36me3,K562_H3K9me3,K562_CTCF,THP-1_DNase,THP-1_H3K4me3,THP-1_H3K27ac,THP-1_H3K27me3,THP-1_H3K4me1,THP-1_H3K36me3,THP-1_H3K9me3,THP-1_CTCF,Jurkat_DNase,Jurkat_H3K4me3,Jurkat_H3K27ac,Jurkat_H3K27me3,Jurkat_H3K4me1,Jurkat_H3K36me3,Jurkat_H3K9me3,Jurkat_CTCF
0,GTCTCTGGGCCGGAAGCCCCCTCTCCTCCCCCGCCGCAGCCCCCTC...,0.565,2.132,1.965,ClassI,0.116,0.016,0.001,0.000,0.001,0.000,0.066,0.001,NaN,0.033,0.016,NaN,NaN,NaN,0.000,0.005,0.140,0.015,0.003,0.106,0.000,0.000,0.001,0.007
1,ACCCGTGGGGAGGCGGTGAACTCCTCCGCAGGGGCCGATGCCCGGG...,-0.362,-0.072,-0.941,ClassI,0.091,0.025,0.001,0.002,0.018,0.000,0.169,0.015,NaN,0.048,0.015,NaN,NaN,NaN,0.002,0.015,0.216,0.036,0.005,0.405,0.000,0.000,0.001,0.039
2,CGCTACCTCACTCACGGGCGAGAACTAAGCACCTACTGTGGACCCG...,0.166,0.504,0.393,ClassI,0.092,0.070,0.009,0.001,0.012,0.000,0.116,0.006,NaN,0.175,0.104,NaN,NaN,NaN,0.001,0.033,0.055,0.052,0.006,0.348,0.000,0.000,0.001,0.016
3,GGGGGCGCTGTGCGGGGCCCAGGCTGCGGCTCCGCTCCCGGCCACG...,-0.423,-1.086,-0.034,ClassIII,0.041,0.023,0.005,0.000,0.002,0.000,0.075,0.192,NaN,0.037,0.044,NaN,NaN,NaN,0.001,0.347,0.071,0.042,0.007,0.127,0.000,0.000,0.000,0.416
4,AACCTGGGACAACAGCAAAAAAAAAAAAAAAAAAAAAAAGGCCCCA...,0.565,0.980,0.799,ClassI,0.242,0.031,0.009,0.005,0.011,0.001,0.081,0.015,NaN,0.063,0.026,NaN,NaN,NaN,0.005,0.041,0.161,0.028,0.009,0.033,0.004,0.001,0.008,0.034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17099,CCCCAAGATTAACCAGTTCTGTCCTGTCAGCAGGAGCAGGCAACTA...,-0.774,5.166,-0.528,ClassI,0.011,0.002,0.002,0.002,0.004,0.001,0.002,0.000,NaN,0.005,0.005,NaN,NaN,NaN,0.001,0.003,0.137,0.005,0.007,0.001,0.013,0.000,0.002,0.004
17100,TTGTTCACGTGTTTACCTGCTGATCTTCTCTCCACTATTATCCTAT...,-0.689,-0.735,-0.553,ClassI,0.004,0.003,0.002,0.002,0.002,0.001,0.006,0.000,NaN,0.002,0.002,NaN,NaN,NaN,0.001,0.001,0.003,0.001,0.000,0.016,0.000,0.000,0.001,0.000
17101,TTTAGTTCTGGATTCTTTTCTTTCGGTACCTCAGACACCTATGAGT...,-0.644,-0.513,-0.332,ClassII,0.158,0.009,0.004,0.002,0.014,0.001,0.008,0.001,NaN,0.038,0.044,NaN,NaN,NaN,0.001,0.026,0.037,0.002,0.001,0.005,0.001,0.000,0.003,0.002
17102,TTTAGCCACTTTCTGCCTAACCTGTCAGAAAGGGTAGATAGCTTCC...,-0.653,-0.642,-0.439,ClassIII,0.035,0.004,0.002,0.004,0.007,0.002,0.004,0.001,NaN,0.007,0.005,NaN,NaN,NaN,0.002,0.004,0.040,0.002,0.001,0.005,0.001,0.000,0.001,0.002


In [3]:
cell_types = ['K562', 'Jurkat']
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']

In [4]:
data_split_dict = {}
data_split_dict['total'] = np.ones(len(MPRA_df), dtype=bool)

# data_split_dict['chrom'] = MPRA_df['chr'].isin([f'chr{i}' for i in (list(range(1, 23)) + ['X', 'Y'])])
# data_split_dict['not_chrom'] = ~data_split_dict['chrom']

# data_split_dict['train'] = MPRA_df['chr'].isin([f'chr{i}' for i in (list(range(3, 23)) + ['X', 'Y'])])
# data_split_dict['valid'] = MPRA_df['chr'].isin(['chr1'])
# data_split_dict['test'] = MPRA_df['chr'].isin(['chr2'])

MPRA_df['std'] = MPRA_df[cell_types].std(axis=1, skipna=True)
threshold = np.percentile(MPRA_df['std'].dropna(), 90)
data_split_dict['diff'] = (MPRA_df['std'] > threshold)
data_split_dict['const'] = ~data_split_dict['diff']


for key in data_split_dict:
    print(key, data_split_dict[key].sum())

keys = list(data_split_dict.keys())
for k1 in keys:
    for k2 in keys:
        data_split_dict[f'{k1}_{k2}'] = data_split_dict[k1] & data_split_dict[k2]

total 17104
diff 1711
const 15393


In [5]:
y_pred = np.load(f'../saved/0507_GosaiMPRA_MyResTransformer3/0507_044041/Reddy_pred.npy')
# y_pred = np.load(f'../saved/0507_GosaiMPRA_MyResTransformer3/0507_044041/Reddy_pred_crop_200bp.npy')

cell_types_pred = [f'{cell_type}_pred' for cell_type in cell_types]
MPRA_df[cell_types_pred] = y_pred
MPRA_df

,seq,Jurkat,K562,THP-1,class,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_H3K27me3,K562_H3K4me1,K562_H3K36me3,K562_H3K9me3,K562_CTCF,THP-1_DNase,THP-1_H3K4me3,THP-1_H3K27ac,THP-1_H3K27me3,THP-1_H3K4me1,THP-1_H3K36me3,THP-1_H3K9me3,THP-1_CTCF,Jurkat_DNase,Jurkat_H3K4me3,Jurkat_H3K27ac,Jurkat_H3K27me3,Jurkat_H3K4me1,Jurkat_H3K36me3,Jurkat_H3K9me3,Jurkat_CTCF,std,K562_pred,Jurkat_pred
0,GTCTCTGGGCCGGAAGCCCCCTCTCCTCCCCCGCCGCAGCCCCCTC...,0.565,2.132,1.965,ClassI,0.116,0.016,0.001,0.000,0.001,0.000,0.066,0.001,NaN,0.033,0.016,NaN,NaN,NaN,0.000,0.005,0.140,0.015,0.003,0.106,0.000,0.000,0.001,0.007,1.108,3.433,3.496
1,ACCCGTGGGGAGGCGGTGAACTCCTCCGCAGGGGCCGATGCCCGGG...,-0.362,-0.072,-0.941,ClassI,0.091,0.025,0.001,0.002,0.018,0.000,0.169,0.015,NaN,0.048,0.015,NaN,NaN,NaN,0.002,0.015,0.216,0.036,0.005,0.405,0.000,0.000,0.001,0.039,0.205,2.839,3.003
2,CGCTACCTCACTCACGGGCGAGAACTAAGCACCTACTGTGGACCCG...,0.166,0.504,0.393,ClassI,0.092,0.070,0.009,0.001,0.012,0.000,0.116,0.006,NaN,0.175,0.104,NaN,NaN,NaN,0.001,0.033,0.055,0.052,0.006,0.348,0.000,0.000,0.001,0.016,0.239,2.694,2.611
3,GGGGGCGCTGTGCGGGGCCCAGGCTGCGGCTCCGCTCCCGGCCACG...,-0.423,-1.086,-0.034,ClassIII,0.041,0.023,0.005,0.000,0.002,0.000,0.075,0.192,NaN,0.037,0.044,NaN,NaN,NaN,0.001,0.347,0.071,0.042,0.007,0.127,0.000,0.000,0.000,0.416,0.469,3.325,3.482
4,AACCTGGGACAACAGCAAAAAAAAAAAAAAAAAAAAAAAGGCCCCA...,0.565,0.980,0.799,ClassI,0.242,0.031,0.009,0.005,0.011,0.001,0.081,0.015,NaN,0.063,0.026,NaN,NaN,NaN,0.005,0.041,0.161,0.028,0.009,0.033,0.004,0.001,0.008,0.034,0.293,3.971,3.927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17099,CCCCAAGATTAACCAGTTCTGTCCTGTCAGCAGGAGCAGGCAACTA...,-0.774,5.166,-0.528,ClassI,0.011,0.002,0.002,0.002,0.004,0.001,0.002,0.000,NaN,0.005,0.005,NaN,NaN,NaN,0.001,0.003,0.137,0.005,0.007,0.001,0.013,0.000,0.002,0.004,4.200,0.221,0.614
17100,TTGTTCACGTGTTTACCTGCTGATCTTCTCTCCACTATTATCCTAT...,-0.689,-0.735,-0.553,ClassI,0.004,0.003,0.002,0.002,0.002,0.001,0.006,0.000,NaN,0.002,0.002,NaN,NaN,NaN,0.001,0.001,0.003,0.001,0.000,0.016,0.000,0.000,0.001,0.000,0.033,0.689,0.478
17101,TTTAGTTCTGGATTCTTTTCTTTCGGTACCTCAGACACCTATGAGT...,-0.644,-0.513,-0.332,ClassII,0.158,0.009,0.004,0.002,0.014,0.001,0.008,0.001,NaN,0.038,0.044,NaN,NaN,NaN,0.001,0.026,0.037,0.002,0.001,0.005,0.001,0.000,0.003,0.002,0.092,2.523,2.058
17102,TTTAGCCACTTTCTGCCTAACCTGTCAGAAAGGGTAGATAGCTTCC...,-0.653,-0.642,-0.439,ClassIII,0.035,0.004,0.002,0.004,0.007,0.002,0.004,0.001,NaN,0.007,0.005,NaN,NaN,NaN,0.002,0.004,0.040,0.002,0.001,0.005,0.001,0.000,0.001,0.002,0.008,-0.707,-0.747


In [6]:
for data_split in ['total', 'const', 'diff']:
    pearson_df = pd.DataFrame(index=cell_types, columns=cell_types_pred)
    for cell_type in cell_types:
        for cell_type_pred in cell_types_pred:
            condition = data_split_dict[data_split]
            df = MPRA_df[condition]
            x = df[f'{cell_type}']
            y = df[f'{cell_type_pred}']
            r, p = pearson(x, y)
            pearson_df.loc[cell_type, cell_type_pred] = r
    print(data_split)
    print(pearson_df)

total
       K562_pred Jurkat_pred
K562       0.637       0.632
Jurkat     0.622       0.629
const
       K562_pred Jurkat_pred
K562       0.641       0.639
Jurkat     0.631       0.637
diff
       K562_pred Jurkat_pred
K562       0.541       0.520
Jurkat     0.487       0.507
